In [1]:
# Options
path = ''
path = '/home/aelt/Dropbox/Apps/ShareLaTeX/peer-2-peer_markets/files/' # path to save files

### Load packages

In [2]:
import numpy as np
import cvxpy as cvx
from print_res_to_latex import *

## Problem data (IEEE14)

In [3]:
n = 14
Pmin = np.array([-21.7,-94.2,-47.8,-7.6,-11.2,-29.5,-9.,-3.5,-6.1,-13.5,-14.9,0.,0.,0.])
Pmax = np.array([0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,20.,50.,10.])
Pren = np.array([0.,0.,0.,14.9,9.,10.,18.5,2.,7.,5.,5.,0.,0.,0.])
a = np.array([4.,1.,2.,10.,8.,4.,9.,15.,18.,7.,6.,7.,2.,8.])
b = np.array([130.,120.,135.,125.,140.,145.,150.,135.,140.,125.,120.,25.,10.,20.])
A = np.diag(a)

## Part 1: Centralized pool-based market

In [41]:
p = cvx.Variable(n)
qimp = cvx.Variable(1)
qexp = cvx.Variable(1)
obj = 1/2 * cvx.quad_form(p, A) + b*p + 70*qimp - 40*qexp
constraints = [
            cvx.sum(p) + qimp - qexp == 0, 
            Pmin + Pren <= p, 
            p <= Pmax + Pren, 
            qimp >= 0, 
            qexp >= 0
            ]
prob = cvx.Problem(cvx.Minimize(obj), constraints)
prob.solve(solver=cvx.MOSEK)
print(prob.value)
print("Schedule: ")
print(p.value)
write_schedule(p,path+'part1_schedule.txt','Schedule for pool-based market')
print("Import: ", qimp.value[0], "\tExport: ", qexp.value[0])
print("Price: ",constraints[0].dual_value[0])
write_pricing(p,constraints[0].dual_value[0],path + 'part1_pricing.txt','Pricing for pool-based market')
print("Payments and revenues under uniform pricing scheme: ")
print(p.value * constraints[0].dual_value[0])
#print("Payments and revenues under uniform pricing scheme (with added renewables): ")
#print((p.value + np.matrix(Pren).T) * constraints[0].dual_value[0])

-3395.572644943103
Schedule: 
[-14.99996698 -50.00015996 -32.49988308   7.29999999  -2.19999997
 -18.74982567   9.49999999  -1.49999994   0.90000001  -7.85700578
  -8.33345189   6.42855887  30.00006739   6.25001133]
Writing table...
Done.
Import:  75.76165567804031 	Export:  -3.1369053391170306e-09
Price:  69.99999987021603
Writing pricing table...
Done.
Payments and revenues under uniform pricing scheme: 
[-1049.99768664 -3500.01119073 -2274.99181139   510.99999848
  -153.99999764 -1312.48779439   664.99999773  -104.99999568
    63.00000076  -549.99040331  -583.3416312    449.99911993
  2100.00471355   437.5007923 ]


#### No external agent:

In [42]:
p = cvx.Variable(n)
obj = 1/2 * cvx.quad_form(p, A) + b*p 
constraints = [
            cvx.sum(p) == 0, 
            Pmin + Pren<= p, 
            p <= Pmax + Pren
            ]
prob = cvx.Problem(cvx.Minimize(obj), constraints)
prob.solve(cvx.MOSEK)
print(prob.value)
print("Schedule: ")
print(p.value) 
write_schedule(p,path + 'part1_schedule_noext.txt','Schedule for pool-based market without external agent')
print("Price: ",constraints[0].dual_value)
print("Payments and revenues under uniform pricing scheme: ")
print(p.value * constraints[0].dual_value)
write_pricing(p,constraints[0].dual_value,path + 'part1_pricing_noext.txt','Pricing for pool-based market without external agent')
#print("Payments and revenues under uniform pricing scheme (with added renewables): ")
#print((p.value + np.matrix(Pren).T) * constraints[0].dual_value)

-2462.983202141747
Schedule: 
[ -8.84543272 -25.38161658 -20.19092832   7.29999997  -2.19999985
 -12.59553371   9.49999994  -1.49999969   0.90000005  -4.34027346
  -4.23029489   9.94558108  42.31105156   9.32744662]
Writing table...
Done.
Price:  94.6189722498422
Payments and revenues under uniform pricing scheme: 
[ -836.94575264 -2401.58247458 -1910.44488629   690.7184944
  -208.16172486 -1191.77645461   898.88023073  -141.92842885
    85.15707943  -410.67221368  -400.26615507   941.04065977
  4003.42821364   882.55341261]
Writing pricing table...
Done.


## Part 2: Community-based market

In [47]:
p = cvx.Variable(n)
q = cvx.Variable(n)
alpha = cvx.Variable(n)
beta = cvx.Variable(n)
qimp = cvx.Variable(1)
qexp = cvx.Variable(1)
obj = 1/2*cvx.quad_form(p, A) + b*p + 70*qimp - 40*qexp
constraints = [
            p + q + alpha - beta == 0, 
            cvx.sum(q) == 0, 
            cvx.sum(alpha) == qimp, 
            cvx.sum(beta) == qexp, 
            Pmin + Pren <= p, 
            p <= Pmax + Pren, 
            qimp >= 0, 
            qexp >= 0, 
            alpha >= 0, 
            beta >= 0
            ]
prob = cvx.Problem(cvx.Minimize(obj), constraints)
prob.solve(cvx.MOSEK)
print(prob.value)
print("Schedule: ")
print(p.value)
write_schedule(p,path + 'part2_schedule.txt','Schedule for community-based market')
print("Community trading: ")
print(q.value)
print("Agent import: ")
print(alpha.value)
print("Agent export: ")
print(beta.value)
print("Import: ", qimp.value, "\tExport: ", qexp.value)
price = abs(constraints[1].dual_value)
print("Price: ",price)
print("Payments and revenues under uniform pricing scheme: ")
print(p.value * price)
write_pricing(p,price,path + 'part2_pricing.txt','Pricing for community-based market')
#print("Payments and revenues under uniform pricing scheme (with added renewables): ")
#print((p.value + np.matrix(Pren).T) * constraints[0].dual_value)

-3395.572644943103
Schedule: 
[-14.99996698 -50.00015996 -32.49988308   7.29999999  -2.19999997
 -18.74982567   9.49999999  -1.49999994   0.90000001  -7.85700578
  -8.33345189   6.42855887  30.00006739   6.25001133]
Writing table...
Done.
Community trading: 
[-60.7616887   50.00015996  32.49988308  -7.29999999   2.19999997
  18.74982567  -9.49999999   1.49999994  -0.90000001   7.85700578
   8.33345189  -6.42855887 -30.00006739  -6.25001133]
Agent import: 
[75.76165568 -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.        ]
Agent export: 
[-3.13690534e-09  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00]
Import:  [75.76165568] 	Export:  [-3.13690534e-09]
Price:  69.99999987021603
Payments and revenues under uniform pricing scheme: 
[-1049.997

## Changing the renewable generation of agent 7

In [48]:
Pren[6] = 0

### Pool-based (no external)

In [51]:
p = cvx.Variable(n)
qimp = cvx.Variable(1)
qexp = cvx.Variable(1)
obj = 1/2*cvx.quad_form(p, A) + b*p
constraints = [
            cvx.sum(p) == 0, 
            Pmin + Pren <= p, 
            p <= Pmax + Pren, 
            qimp >= 0, 
            qexp >= 0
            ]
prob = cvx.Problem(cvx.Minimize(obj), constraints)
prob.solve(cvx.MOSEK)
print(prob.value)
print("Schedule: ")
print(p.value)
write_schedule(p,path + 'part1_schedule_a7changed.txt','Schedule for pool-based market without agent 7\'s production')
print("Import: ", qimp.value, "\tExport: ", qexp.value)
print("Price: ",constraints[0].dual_value)
print("Payments and revenues under uniform pricing scheme: ")
print(p.value * constraints[0].dual_value)
write_pricing(p,constraints[0].dual_value,path + 'part1_pricing_a7changed.txt','Pricing for pool-based market without agent 7\'s production')

-3527.196100469998
Schedule: 
[ -7.61793949 -20.4717585  -17.73587989   7.3         -2.19999998
 -11.36794101  -5.60797393  -1.49999996   0.90000001  -3.63882333
  -3.41196017  10.64689224  44.76413608   9.94124795]
Writing table...
Done.
Import:  [0.] 	Export:  [0.]
Price:  99.5283142281432
Payments and revenues under uniform pricing scheme: 
[ -758.20067541 -2037.51961264 -1765.22222713   726.55669355
  -218.96228926 -1131.43200448  -558.15219158  -149.29246729
    89.57548339  -362.16595188  -339.5866442   1059.66723601
  4455.29900151   989.43564941]
Writing pricing table...
Done.


### Pool-based

In [54]:
p = cvx.Variable(n)
qimp = cvx.Variable(1)
qexp = cvx.Variable(1)
obj = 1/2*cvx.quad_form(p, A) + b*p + 70*qimp - 40*qexp
constraints = [
            cvx.sum(p) + qimp - qexp == 0, 
            Pmin + Pren <= p, 
            p <= Pmax + Pren, 
            qimp >= 0, 
            qexp >= 0
            ]
prob = cvx.Problem(cvx.Minimize(obj), constraints)
prob.solve(cvx.MOSEK)
print(prob.value)
print("Schedule: ")
print(p.value)
write_schedule(p,path + 'part1_schedule_a7changed.txt','Schedule for pool-based market without agent 7\'s production')
print("Import: ", qimp.value, "\tExport: ", qexp.value)
price = abs(constraints[0].dual_value[0])
print("Price: ",price)
print("Payments and revenues under uniform pricing scheme: ")
print(p.value * price)
write_pricing(p,price,path + 'part1_pricing_a7changed.txt','Pricing for pool-based market without agent 7\'s production')

-4917.253222421599
Schedule: 
[-14.99999992 -50.00000014 -32.49999997   7.29999998  -2.19999994
 -18.74999811  -8.88885207  -1.49999992   0.90000002  -7.8571416
  -8.33333256   6.42857155  29.99999992   6.24999987]
Writing table...
Done.
Import:  [94.15075289] 	Export:  [1.80407967e-09]
Price:  69.99999974255782
Payments and revenues under uniform pricing scheme: 
[-1049.9999907  -3499.99999726 -2274.99998933   510.99999689
  -153.99999554 -1312.49986268  -622.21964238  -104.99999367
    63.00000143  -549.9999101   -583.33327734   450.0000071
  2099.99998669   437.49998951]
Writing pricing table...
Done.


### Community-based:

In [57]:
p = cvx.Variable(n)
q = cvx.Variable(n)
alpha = cvx.Variable(n)
beta = cvx.Variable(n)
qimp = cvx.Variable(1)
qexp = cvx.Variable(1)
obj = 1/2*cvx.quad_form(p, A) + b*p + 70*qimp - 40*qexp
constraints = [
            p + q + alpha - beta == 0, 
            cvx.sum(q) == 0, 
            cvx.sum(alpha) == qimp, 
            cvx.sum(beta) == qexp, 
            Pmin + Pren <= p, 
            p <= Pmax + Pren, 
            qimp >= 0, 
            qexp >= 0, 
            alpha >= 0, 
            beta >= 0
            ]
prob = cvx.Problem(cvx.Minimize(obj), constraints)
prob.solve(cvx.MOSEK)
print(prob.value)
print("Schedule: ")
print(p.value)
write_schedule(p,path + 'part2_schedule_a7changed.txt','Schedule for community-based market without agent 7\'s production')
print("Community trading: ")
print(q.value)
print("Agent import: ")
print(alpha.value)
print("Agent export: ")
print(beta.value)
print("Import: ", qimp.value, "\tExport: ", qexp.value)
price = abs(constraints[1].dual_value)
print("Price: ",price)
print("Payments and revenues under uniform pricing scheme: ")
print(p.value * price)
write_pricing(p,price,path + 'part2_pricing_a7changed.txt','Pricing for community-based market without agent 7\'s production')
#print("Payments and revenues under uniform pricing scheme (with added renewables): ")
#print((p.value + np.matrix(Pren).T) * constraints[0].dual_value)

-4917.253226093274
Schedule: 
[-14.99999993 -50.00000016 -32.49999998   7.29999998  -2.19999995
 -18.74999811  -8.88885208  -1.49999992   0.90000002  -7.85714161
  -8.33333257   6.42857156  29.99999993   6.24999988]
Writing table...
Done.
Community trading: 
[  14.99999993   50.00000016   32.49999998   -7.29999998    2.19999995
   18.74999811    8.88885208    1.49999992   -0.90000002    7.85714161
    8.33333257   -6.42857156 -124.15075284   -6.24999988]
Agent import: 
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 94.15075291  0.        ]
Agent export: 
[0.00000000e+00 1.80562469e-09 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
Import:  [94.15075291] 	Export:  [1.80562469e-09]
Price:  69.99999974265384
Payments and revenues under uniform pricing scheme: 
[-1049.9999

# ADMM

In [ ]:
def admm_community_based(p,q,alpha,beta,Pmax,Pmin,rhoexp = 40, tau = 30):
    '''
    Solves community-based market in a distributed manner 
    using alternating direction method of multipliers (ADMM).
    
    Input: 
        Feasible initial guesses for variables
        p:     power production (consumption of negative)
        q:     energy import from community
        alpha: energy imported
        beta:  energy exported
        
        Parameters: 
        Pmax:   maximum production
        Pmin:   minimum production
        rhoexp: price for energy exported to outside
        tau:    added price (to export price) for energy import 
        
    Output: 
        The variables
    '''
    
    